In [9]:
class Substitution:
    def __init__(self):
        self.score = []

    def build_score(self, residues, residuescores):
        self.score = [[0 for _ in range(127)] for _ in range(127)]
        for i in range(len(residues)):
            res1 = residues[i]
            for j in range(i + 1):
                res2 = residues[j]
                self.score[ord(res1)][ord(res2)] = self.score[ord(res2)][ord(res1)] \
                                                 = self.score[ord(res1)][ord(res2) + 32] \
                                                 = self.score[ord(res2) + 32][ord(res1)] \
                                                 = self.score[ord(res1) + 32][ord(res2)] \
                                                 = self.score[ord(res2)][ord(res1) + 32] \
                                                 = self.score[ord(res1) + 32][ord(res2) + 32] \
                                                 = self.score[ord(res2) + 32][ord(res1) + 32] \
                                                 = residuescores[i][j]

    def get_residues(self):
        pass

class Blosum50(Substitution):
    
    def __init__(self):
        self.residuescores = [
            # A  R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V
            [  5],
            [-2, 7],
            [-1,-1, 7],
            [-2,-2, 2, 8],
            [-1,-4,-2,-4,13],
            [-1, 1, 0, 0,-3, 7],
            [-1, 0, 0, 2,-3, 2, 6],
            [  0,-3, 0,-1,-3,-2,-3, 8],
            [-2, 0, 1,-1,-3, 1, 0,-2,10],
            [-1,-4,-3,-4,-2,-3,-4,-4,-4, 5],
            [-2,-3,-4,-4,-2,-2,-3,-4,-3, 2, 5],
            [-1, 3, 0,-1,-3, 2, 1,-2, 0,-3,-3, 6],
            [-1,-2,-2,-4,-2, 0,-2,-3,-1, 2, 3,-2, 7],
            [-3,-3,-4,-5,-2,-4,-3,-4,-1, 0, 1,-4, 0, 8],
            [-1,-3,-2,-1,-4,-1,-1,-2,-2,-3,-4,-1,-3,-4,10],
            [  1,-1, 1, 0,-1, 0,-1, 0,-1,-3,-3, 0,-2,-3,-1, 5],
            [  0,-1, 0,-1,-1,-1,-1,-2,-2,-1,-1,-1,-1,-2,-1, 2, 5],
            [-3,-3,-4,-5,-5,-1,-3,-3,-3,-3,-2,-3,-1, 1,-4,-4,-3,15],
            [-2,-1,-2,-3,-3,-1,-2,-3, 2,-1,-1,-2, 0, 4,-3,-2,-2, 2, 8],
            [  0,-3,-3,-4,-1,-3,-3,-4,-4, 4, 1,-3, 1,-1,-3,-2, 0,-3,-1, 5]
        ]
        self.residues = "ARNDCQEGHILKMFPSTWYV"
        super().build_score(self.residues, self.residuescores)
    
    def get_residues(self):
        return self.residues

class Align:
    NegInf = float("-inf")

    def __init__(self, sub, d, seq1, seq2):
        self.sub = sub
        self.seq1 = self.strip(seq1)
        self.seq2 = self.strip(seq2)
        self.d = d
        self.n = len(self.seq1)
        self.m = len(self.seq2)
        self.B0 = None

    def strip(self, s):
        valid = [False] * 127
        residues = self.sub.get_residues()
        for i in range(len(residues)):
            c = residues[i]
            if ord(c) < 96:
                valid[ord(c)] = valid[ord(c) + 32] = True
            else:
                valid[ord(c) - 32] = valid[ord(c)] = True
        res = []
        for i in range(len(s)):
            if valid[ord(s[i])]:
                res.append(s[i])
        return ''.join(res)

    def get_match(self):
        res1 = []
        res2 = []
        tb = self.B0
        i = tb.i
        j = tb.j
        while tb is not None:
            if i == tb.i:
                res1.append('-')
            else:
                res1.append(self.seq1[i - 1])
            if j == tb.j:
                res2.append('-')
            else:
                res2.append(self.seq2[j - 1])
            i = tb.i
            j = tb.j
            tb = self.next(tb)
        return [''.join(res1[::-1]), ''.join(res2[::-1])]

    def fmt_score(self, val):
        if val < self.NegInf / 2:
            return "-Inf"
        else:
            return str(val)

    def do_match(self, out, msg, udskrivF=True):
        print(msg + ":")
        print("Score = " + str(self.get_score()))
        if udskrivF:
            print("The F matrix:")
            self.printf(out)
        print("An optimal alignment:")
        match = self.get_match()
        print(match[0])
        print(match[1])
        print()

    def next(self, tb):
        return tb

    def get_score(self):
        pass

    def printf(self, out):
        pass

    '''@staticmethod
    def max(x1, x2):
        return max(x1, x2)

    @staticmethod
    def max_2(x1, x2, x3):
        return max(x1, max(x2, x3))

    @staticmethod
    def max_3(x1, x2, x3, x4):
        return max(Align.max(x1, x2), Align.max(x3, x4))'''
    def max(x1, x2, x3=None, x4=None):
        if x3 is None and x4 is None:
            return max(x1, x2)
        elif x4 is None:
            return max(x1, max(x2, x3))
        else:
            return max(max(x1, x2), max(x3, x4))

    @staticmethod
    def pad_left(s, width):
        filler = width - len(s)
        if filler > 0:
            return ' ' * filler + s
        else:
            return s

class AlignSimple(Align):
    def __init__(self, sub, d, seq1, seq2):
        super().__init__(sub, d, seq1, seq2)
        self.F = [[0] * (self.m + 1) for _ in range(self.n + 1)]
        self.B = [[None] * (self.m + 1) for _ in range(self.n + 1)]

    def next(self, tb):
        tb2 = tb
        return self.B[tb2.i][tb2.j]

    def get_score(self):
        return self.F[self.B0.i][self.B0.j]

    def printf(self, out):
        for j in range(self.m + 1):
            for i in range(len(self.F)):
                out.print(Align.pad_left(self.fmt_score(self.F[i][j]), 5))
            print()

class Traceback:
    def __init__(self):
        self.i = 0
        self.j = 0

class Traceback2(Traceback):
    def __init__(self, i, j):
        super().__init__()
        self.i = i
        self.j = j

class Output:
    def print(self, s):
        pass

    def println(self, s):
        pass

    def println(self):
        pass

class SystemOut(Output):
    def print(self, s):
        print(s, end='')

    def println(self, s):
        print(s)

    def println(self):
        print()

class NW(AlignSimple):
    def __init__(self, sub, d, seq1, seq2):
        super().__init__(sub, d, seq1, seq2)
        score = sub.score
        for i in range(1, self.n + 1):
            self.F[i][0] = -d * i
            self.B[i][0] = Traceback2(i - 1, 0)
        for j in range(1, self.m + 1):
            self.F[0][j] = -d * j
            self.B[0][j] = Traceback2(0, j - 1)
        for i in range(1, self.n + 1):
            for j in range(1, self.m + 1):
                s = score[ord(self.seq1[i - 1])][ord(self.seq2[j - 1])]
                val = Align.max(self.F[i - 1][j - 1] + s, self.F[i - 1][j] - d, self.F[i][j - 1] - d)
                self.F[i][j] = val
                if val == self.F[i - 1][j - 1] + s:
                    self.B[i][j] = Traceback2(i - 1, j - 1)
                elif val == self.F[i - 1][j] - d:
                    self.B[i][j] = Traceback2(i - 1, j)
                elif val == self.F[i][j - 1] - d:
                    self.B[i][j] = Traceback2(i, j - 1)
                else:
                    raise Exception("NW 1")
        self.B0 = Traceback2(self.n, self.m)

class Match2:
    @staticmethod
    def main(args):
        out = SystemOut()
        seq1 = args[0]
        seq2 = args[1]
        sub = Blosum50()
        nw = NW(sub, 8, seq1, seq2)
        nw.do_match(out, "GLOBAL ALIGNMENT")

if __name__ == "__main__":
    Match2.main(["HEAGAWGHEE", "PAWHEAE"])

GLOBAL ALIGNMENT:
Score = 1
The F matrix:
    0   -8  -16  -24  -32  -40  -48  -56  -64  -72  -80
   -8   -2   -9  -17  -25  -33  -41  -49  -57  -65  -73
  -16  -10   -3   -4  -12  -20  -28  -36  -44  -52  -60
  -24  -18  -11   -6   -7  -15   -5  -13  -21  -29  -37
  -32  -14  -18  -13   -8   -9  -13   -7   -3  -11  -19
  -40  -22   -8  -16  -16   -9  -12  -15   -7    3   -5
  -48  -30  -16   -3  -11  -11  -12  -12  -15   -5    2
  -56  -38  -24  -11   -6  -12  -14  -15  -12   -9    1
An optimal alignment:
HEAGAWGHE-E-
--P-AW-HEAE-



In [10]:
class SubstitutionMatrix:
    def __init__(self):
        self.matrix = [[0] * 127 for _ in range(127)]


    def build_matrix(self, residues, scores):
        char_to_ord = {c: ord(c) for c in residues}
        for i, res1 in enumerate(residues):
            ord_res1 = char_to_ord[res1]
            for j, res2 in enumerate(residues[:i + 1]):
                ord_res2 = char_to_ord[res2]
                self.matrix[ord_res1][ord_res2] = self.matrix[ord_res2][ord_res1] = \
                    self.matrix[ord_res1][ord_res2 + 32] = self.matrix[ord_res2 + 32][ord_res1] = \
                    self.matrix[ord_res1 + 32][ord_res2] = self.matrix[ord_res2][ord_res1 + 32] = \
                    self.matrix[ord_res1 + 32][ord_res2 + 32] = scores[i][j]


    def get_residues(self):
        pass




class Blosum50Matrix(SubstitutionMatrix):


    def __init__(self):
        self.scores = [
            # A  R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V
            [5],
            [-2, 7],
            [-1, -1, 7],
            [-2, -2, 2, 8],
            [-1, -4, -2, -4, 13],
            [-1, 1, 0, 0, -3, 7],
            [-1, 0, 0, 2, -3, 2, 6],
            [0, -3, 0, -1, -3, -2, -3, 8],
            [-2, 0, 1, -1, -3, 1, 0, -2, 10],
            [-1, -4, -3, -4, -2, -3, -4, -4, -4, 5],
            [-2, -3, -4, -4, -2, -2, -3, -4, -3, 2, 5],
            [-1, 3, 0, -1, -3, 2, 1, -2, 0, -3, -3, 6],
            [-1, -2, -2, -4, -2, 0, -2, -3, -1, 2, 3, -2, 7],
            [-3, -3, -4, -5, -2, -4, -3, -4, -1, 0, 1, -4, 0, 8],
            [-1, -3, -2, -1, -4, -1, -1, -2, -2, -3, -4, -1, -3, -4, 10],
            [1, -1, 1, 0, -1, 0, -1, 0, -1, -3, -3, 0, -2, -3, -1, 5],
            [0, -1, 0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 2, 5],
            [-3, -3, -4, -5, -5, -1, -3, -3, -3, -3, -2, -3, -1, 1, -4, -4, -3, 15],
            [-2, -1, -2, -3, -3, -1, -2, -3, 2, -1, -1, -2, 0, 4, -3, -2, -2, 2, 8],
            [0, -3, -3, -4, -1, -3, -3, -4, -4, 4, 1, -3, 1, -1, -3, -2, 0, -3, -1, 5]
        ]
        self.residues = "ARNDCQEGHILKMFPSTWYV"  # amino acid sequence
        super().__init__()
        super().build_matrix(self.residues, self.scores)


    def get_residues(self):
        return self.residues




class Alignment:
    NegInf = float("-inf")


    def __init__(self, matrix, gap_penalty, sequence1, sequence2):
        self.matrix = matrix
        self.seq1 = self.strip(sequence1)
        self.seq2 = self.strip(sequence2)
        self.gap_penalty = gap_penalty
        self.n = len(self.seq1)
        self.m = len(self.seq2)
        self.B0 = None


    def strip(self, sequence):
        valid = [False] * 127
        residues = self.matrix.get_residues()
        for i in range(len(residues)):
            c = residues[i]
            if ord(c) < 96:
                valid[ord(c)] = valid[ord(c) + 32] = True
            else:
                valid[ord(c) - 32] = valid[ord(c)] = True
        res = []
        for i in range(len(sequence)):
            if valid[ord(sequence[i])]:
                res.append(sequence[i])
        return ''.join(res)


    def get_alignment(self):
        res1 = []
        res2 = []
        tb = self.B0
        i = tb.i
        j = tb.j
        while tb is not None:
            if i == tb.i:
                res1.append('-')
            else:
                res1.append(self.seq1[i - 1])
            if j == tb.j:
                res2.append('-')
            else:
                res2.append(self.seq2[j - 1])
            i = tb.i
            j = tb.j
            tb = self.next_traceback(tb)
        return [''.join(res1[::-1]), ''.join(res2[::-1])]


    def format_score(self, val):
        if val < self.NegInf / 2:
            return "-Inf"
        else:
            return str(val)


    def calculate_alignment(self, output, message, print_matrix=True):
        print(message + ":")
        print()
        print("Maximum Global Alignment Score = " + str(self.get_alignment_score()))
        if print_matrix:
            print("The alignment matrix:")
            self.print_matrix(output)
        print()
        print("An optimal alignment:")
        alignment = self.get_alignment()
        print(alignment[0])
        print(alignment[1])
        print()


    def next_traceback(self, traceback):
        return traceback


    def get_alignment_score(self):
        pass


    def print_matrix(self, output):
        pass


    @staticmethod
    def max_value(x1, x2, x3=None, x4=None):
        if x3 is None and x4 is None:
            return max(x1, x2)
        elif x4 is None:
            return max(x1, max(x2, x3))
        else:
            return max(max(x1, x2), max(x3, x4))


    @staticmethod
    def pad_left(s, width):
        filler = width - len(s)
        if filler > 0:
            return ' ' * filler + s
        else:
            return s




class SimpleAlignment(Alignment):
    def __init__(self, matrix, gap_penalty, sequence1, sequence2):
        super().__init__(matrix, gap_penalty, sequence1, sequence2)
        self.score_matrix = [[0] * (self.m + 1) for _ in range(self.n + 1)]
        self.traceback_matrix = [[None] * (self.m + 1) for _ in range(self.n + 1)]


    def next_traceback(self, traceback):
        traceback2 = traceback
        return self.traceback_matrix[traceback2.i][traceback2.j]


    def get_alignment_score(self):
        return self.score_matrix[self.B0.i][self.B0.j]


    def print_matrix(self, output):
        for j in range(self.m + 1):
            for i in range(len(self.score_matrix)):
                output.print_output(self.pad_left(self.format_score(self.score_matrix[i][j]), 5))
            print()




class TracebackNode:
    def __init__(self, i=0, j=0):
        self.i = i
        self.j = j


class TracebackNode2(TracebackNode):
    def __init__(self, i, j):
        super().__init__()
        self.i = i
        self.j = j


class OutputHandler:
    def print_output(self, text):
        pass


    def println_output(self, text):
        pass


    def println(self):
        pass


class ConsoleOutputHandler(OutputHandler):
    def print_output(self, text):
        print(text, end='')


    def println_output(self, text):
        print(text)


    def println(self):
        print()


class NeedlemanWunschAlignment(SimpleAlignment):
    def __init__(self, matrix, gap_penalty, sequence1, sequence2):
        super().__init__(matrix, gap_penalty, sequence1, sequence2)
        scores = matrix.matrix
        for i in range(1, self.n + 1):
            self.score_matrix[i][0] = -gap_penalty * i
            self.traceback_matrix[i][0] = TracebackNode2(i - 1, 0)
        for j in range(1, self.m + 1):
            self.score_matrix[0][j] = -gap_penalty * j
            self.traceback_matrix[0][j] = TracebackNode2(0, j - 1)
        for i in range(1, self.n + 1):
            for j in range(1, self.m + 1):
                score = scores[ord(self.seq1[i - 1])][ord(self.seq2[j - 1])]
                val = self.max_value(
                    self.score_matrix[i - 1][j - 1] + score,
                    self.score_matrix[i - 1][j] - gap_penalty,
                    self.score_matrix[i][j - 1] - gap_penalty
                )
                self.score_matrix[i][j] = val
                if val == self.score_matrix[i - 1][j - 1] + score:
                    self.traceback_matrix[i][j] = TracebackNode2(i - 1, j - 1)
                elif val == self.score_matrix[i - 1][j] - gap_penalty:
                    self.traceback_matrix[i][j] = TracebackNode2(i - 1, j)
                elif val == self.score_matrix[i][j - 1] - gap_penalty:
                    self.traceback_matrix[i][j] = TracebackNode2(i, j - 1)
                else:
                    raise Exception("Needleman-Wunsch 1")
        self.B0 = TracebackNode2(self.n, self.m)


class HirschbergAlignment(Alignment):
    def __init__(self, matrix, gap_penalty, sequence1, sequence2):
        super().__init__(matrix, gap_penalty, sequence1, sequence2)
        self.score_matrix = [[0] * (self.m + 1) for _ in range(2)]
        self.traceback_matrix = [[None] * (self.m + 1) for _ in range(2)]


    def next_traceback(self, traceback):
        traceback2 = traceback
        return self.traceback_matrix[traceback2.i % 2][traceback2.j]


    def get_alignment_score(self):
        return self.score_matrix[self.n % 2][self.m]


    def print_matrix(self, output):
        for j in range(self.m + 1):
            for i in range(2):
                output.print_output(self.pad_left(self.format_score(self.score_matrix[i][j]), 5))
            print()


    def hirschberg(self, seq1, seq2):
        self.seq1 = seq1
        self.seq2 = seq2
        self.n = len(seq1)
        self.m = len(seq2)


        if self.n == 0:
            return ["-" * self.m, seq2]
        if self.m == 0:
            return [seq1, "-" * self.n]
        if self.n == 1 or self.m == 1:
            return self.needleman_wunsch()


        middle = self.n // 2
        row1 = self.align(seq1[:middle], seq2)
        row2 = self.align(seq1[middle:][::-1], seq2[::-1])
        row2[0] = row2[0][::-1]
        row2[1] = row2[1][::-1]


        result = [row1[0] + row2[0], row1[1] + row2[1]]
        return result


    def align(self, seq1, seq2):
        self.seq1 = seq1
        self.seq2 = seq2
        self.n = len(seq1)
        self.m = len(seq2)


        for j in range(self.m + 1):
            self.score_matrix[0][j] = -self.gap_penalty * j
            self.traceback_matrix[0][j] = TracebackNode2(0, j - 1)


        for i in range(1, self.n + 1):
            self.score_matrix[i % 2][0] = -self.gap_penalty * i
            self.traceback_matrix[i % 2][0] = TracebackNode2((i - 1) % 2, 0)
            for j in range(1, self.m + 1):
                score = self.matrix.matrix[ord(seq1[i - 1])][ord(seq2[j - 1])]
                val = self.max_value(
                    self.score_matrix[(i - 1) % 2][j - 1] + score,
                    self.score_matrix[(i - 1) % 2][j] - self.gap_penalty,
                    self.score_matrix[i % 2][j - 1] - self.gap_penalty
                )
                self.score_matrix[i % 2][j] = val
                if val == self.score_matrix[(i - 1) % 2][j - 1] + score:
                    self.traceback_matrix[i % 2][j] = TracebackNode2((i - 1) % 2, j - 1)
                elif val == self.score_matrix[(i - 1) % 2][j] - self.gap_penalty:
                    self.traceback_matrix[i % 2][j] = TracebackNode2((i - 1) % 2, j)
                elif val == self.score_matrix[i % 2][j - 1] - self.gap_penalty:
                    self.traceback_matrix[i % 2][j] = TracebackNode2(i % 2, j - 1)
                else:
                    raise Exception("Hirschberg Alignment")


        return [seq1, seq2]




class SequenceAlignment:
    @staticmethod
    def main(args):
        output_handler = ConsoleOutputHandler()
        sequence1 = args[0]
        sequence2 = args[1]


        matrix = Blosum50Matrix()


        if len(sequence1) < 500 and len(sequence2) < 500:
            # Use Hirschberg algorithm for shorter sequences
            hirschberg_alignment = HirschbergAlignment(matrix, 8, sequence1, sequence2)
            alignment_result = hirschberg_alignment.hirschberg(sequence1, sequence2)
        else:
            # Use Needleman-Wunsch for longer sequences
            nw_alignment = NeedlemanWunschAlignment(matrix, 8, sequence1, sequence2)
            alignment_result = nw_alignment.calculate_alignment(output_handler, "GLOBAL ALIGNMENT", False)


        # Print the alignment score from alignment_result
        print("Maximum Global Alignment Score =", alignment_result[0])


        print("The alignment matrix:")
        print(alignment_result[1])  # Fix this line
        print()
        print("An optimal alignment:")
        alignment = alignment_result[1].get_alignment()
        print(alignment[0])
        print(alignment[1])
        print()


if __name__ == "__main__":
    SequenceAlignment.main(["HEAGAWGHEE", "PAWHEAE"])


Maximum Global Alignment Score = HEAGAWGHEE
The alignment matrix:
PAWHEAEPAWHEAE

An optimal alignment:


AttributeError: 'str' object has no attribute 'get_alignment'

In [13]:
class Substitution:
    def __init__(self):
        self.score = []

    def build_score(self, residues, residuescores):
        self.score = [[0 for _ in range(127)] for _ in range(127)]
        for i in range(len(residues)):
            res1 = residues[i]
            for j in range(i + 1):
                res2 = residues[j]
                self.score[ord(res1)][ord(res2)] = self.score[ord(res2)][ord(res1)] \
                                                 = self.score[ord(res1)][ord(res2) + 32] \
                                                 = self.score[ord(res2) + 32][ord(res1)] \
                                                 = self.score[ord(res1) + 32][ord(res2)] \
                                                 = self.score[ord(res2)][ord(res1) + 32] \
                                                 = self.score[ord(res1) + 32][ord(res2) + 32] \
                                                 = self.score[ord(res2) + 32][ord(res1) + 32] \
                                                 = residuescores[i][j]

    def get_residues(self):
        pass

class Blosum50(Substitution):
    
    def __init__(self):
        self.residuescores = [
            # A  R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V
            [  5],
            [-2, 7],
            [-1,-1, 7],
            [-2,-2, 2, 8],
            [-1,-4,-2,-4,13],
            [-1, 1, 0, 0,-3, 7],
            [-1, 0, 0, 2,-3, 2, 6],
            [  0,-3, 0,-1,-3,-2,-3, 8],
            [-2, 0, 1,-1,-3, 1, 0,-2,10],
            [-1,-4,-3,-4,-2,-3,-4,-4,-4, 5],
            [-2,-3,-4,-4,-2,-2,-3,-4,-3, 2, 5],
            [-1, 3, 0,-1,-3, 2, 1,-2, 0,-3,-3, 6],
            [-1,-2,-2,-4,-2, 0,-2,-3,-1, 2, 3,-2, 7],
            [-3,-3,-4,-5,-2,-4,-3,-4,-1, 0, 1,-4, 0, 8],
            [-1,-3,-2,-1,-4,-1,-1,-2,-2,-3,-4,-1,-3,-4,10],
            [  1,-1, 1, 0,-1, 0,-1, 0,-1,-3,-3, 0,-2,-3,-1, 5],
            [  0,-1, 0,-1,-1,-1,-1,-2,-2,-1,-1,-1,-1,-2,-1, 2, 5],
            [-3,-3,-4,-5,-5,-1,-3,-3,-3,-3,-2,-3,-1, 1,-4,-4,-3,15],
            [-2,-1,-2,-3,-3,-1,-2,-3, 2,-1,-1,-2, 0, 4,-3,-2,-2, 2, 8],
            [  0,-3,-3,-4,-1,-3,-3,-4,-4, 4, 1,-3, 1,-1,-3,-2, 0,-3,-1, 5]
        ]
        self.residues = "ARNDCQEGHILKMFPSTWYV"
        super().build_score(self.residues, self.residuescores)
    
    def get_residues(self):
        return self.residues

class Align:
    NegInf = float("-inf")

    def __init__(self, sub, d, seq1, seq2):
        self.sub = sub
        self.seq1 = self.strip(seq1)
        self.seq2 = self.strip(seq2)
        self.d = d
        self.n = len(self.seq1)
        self.m = len(self.seq2)
        self.B0 = None

    def strip(self, s):
        valid = [False] * 127
        residues = self.sub.get_residues()
        for i in range(len(residues)):
            c = residues[i]
            if ord(c) < 96:
                valid[ord(c)] = valid[ord(c) + 32] = True
            else:
                valid[ord(c) - 32] = valid[ord(c)] = True
        res = []
        for i in range(len(s)):
            if valid[ord(s[i])]:
                res.append(s[i])
        return ''.join(res)

    def get_match(self):
        res1 = []
        res2 = []
        tb = self.B0
        i = tb.i
        j = tb.j
        while tb is not None:
            if i == tb.i:
                res1.append('-')
            else:
                res1.append(self.seq1[i - 1])
            if j == tb.j:
                res2.append('-')
            else:
                res2.append(self.seq2[j - 1])
            i = tb.i
            j = tb.j
            tb = self.next(tb)
        return [''.join(res1[::-1]), ''.join(res2[::-1])]

    def fmt_score(self, val):
        if val < self.NegInf / 2:
            return "-Inf"
        else:
            return str(val)

    def do_match(self, out, msg, udskrivF=True):
        print(msg + ":")
        print("Score = " + str(self.get_score()))
        if udskrivF:
            print("The F matrix:")
            self.printf(out)
        print("An optimal alignment:")
        match = self.get_match()
        print(match[0])
        print(match[1])
        print()

    def next(self, tb):
        return tb

    def get_score(self):
        pass

    def printf(self, out):
        pass

    @staticmethod
    def pad_left(s, width):
        filler = width - len(s)
        if filler > 0:
            return ' ' * filler + s
        else:
            return s

class AlignSimple(Align):
    def __init__(self, sub, d, seq1, seq2):
        super().__init__(sub, d, seq1, seq2)
        self.F = [[0] * (self.m + 1) for _ in range(2)]  # Use only two rows

    def next(self, tb):
        tb2 = tb
        return self.B[tb2.i][tb2.j]

    def get_score(self):
        return self.F[1][self.B0.j]

    def printf(self, out):
        for j in range(self.m + 1):
            out.print(Align.pad_left(self.fmt_score(self.F[1][j]), 5))
        print()

class Traceback:
    def __init__(self):
        self.i = 0
        self.j = 0

class Traceback2(Traceback):
    def __init__(self, i, j):
        super().__init__()
        self.i = i
        self.j = j

class Output:
    def print(self, s):
        pass

    def println(self, s):
        pass

    def println(self):
        pass

class SystemOut(Output):
    def print(self, s):
        print(s, end='')

    def println(self, s):
        print(s)

    def println(self):
        print()

class Hirschberg(AlignSimple):
    def __init__(self, sub, d, seq1, seq2):
        super().__init__(sub, d, seq1, seq2)
        score = sub.score
        n = self.n
        m = self.m
        F0 = [0] * (m + 1)
        F1 = [0] * (m + 1)
        self.F = [F0, F1]

        # Calculate the middle row index
        middle = n // 2

        # Calculate the two halves of the sequences
        seq1_half1, seq1_half2 = seq1[:middle], seq1[middle:]
        seq2_half1, seq2_half2 = seq2, seq2[::-1]  # Reverse seq2 for alignment

        # Align the two halves separately
        F0[0] = 0
        for j in range(1, m + 1):
            F0[j] = F0[j - 1] - d

        for i in range(1, len(seq1_half1) + 1):
            F1[0] = F0[0] - d
            for j in range(1, m + 1):
                s = score[ord(seq1_half1[i - 1])][ord(seq2_half1[j - 1])]
                val = Align.max(F0[j - 1] + s, F0[j] - d, F1[j - 1] - d)
                F1[j] = val
            F0, F1 = F1, F0

        F0[-1] = 0
        for j in range(m - 1, -1, -1):
            F0[j] = F0[j + 1] - d

        F2 = [0] * (m + 1)
        self.B = [F0, F1, F2]

        for i in range(1, len(seq1_half2) + 1):
            F2[0] = F0[0] - d
            for j in range(1, m + 1):
                s = score[ord(seq1_half2[i - 1])][ord(seq2_half2[j - 1])]
                val = Align.max(F0[j - 1] + s, F0[j] - d, F2[j - 1] - d)
                F2[j] = val
            F0, F2 = F2, F0

        self.B0 = Traceback2(middle, m)

if __name__ == "__main__":
    out = SystemOut()
    seq1 = "HEAGAWGHEE"
    seq2 = "PAWHEAE"
    sub = Blosum50()
    hirschberg = Hirschberg(sub, 8, seq1, seq2)
    hirschberg.do_match(out, "GLOBAL ALIGNMENT")


AttributeError: type object 'Align' has no attribute 'max'